# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import os
os.chdir('..')

import logging

# Set logging level to suppress warnings
logging.getLogger().setLevel(logging.ERROR)
# Just disables the warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

from nn.models_tf2 import Models
from botbidder import BotBid
from util import hand_to_str
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)

models = Models.from_conf(conf.load('./config/default.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('./config/default.conf'), False)  # Load sampling strategies


Loading config file d:\GitHub\ben\src\./config/default.conf
Loading config file d:\GitHub\ben\src\./config/default.conf


### Running through an example

In [2]:
# East deals, EW vulnerable.
vuln_ns, vuln_ew = True, True

from ddsolver import ddsolver
dds = ddsolver.DDSolver()

# you sit West and hold:
hand = '843.9765.A73.AK4'

# the auction goes:
auction = []
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, 0, 0, dds, False, False)
bid = bot_bid.bid(auction)
print(bid.to_dict()['candidates'])

auction = ["PAD_START"]
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, 1, 1, dds, False, False)
bid = bot_bid.bid(auction)
print(bid.to_dict()['candidates'])

auction = ["PAD_START","PAD_START"]
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, 2, 2, dds, False, False)
bid = bot_bid.bid(auction)
print(bid.to_dict()['candidates'])

auction = ["PAD_START","PAD_START","PAD_START"]
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, 3, 3, dds, False, False)
bid = bot_bid.bid(auction)
print(bid.to_dict()['candidates'])

# what's your bid?

[{'call': '1C', 'insta_score': 1.198, 'alert': 'False', 'explanation': '3+ !C -- 3+!C; 6-!D; 5-!H; 5-!S; 11-21 HCP'}]
[{'call': '1C', 'insta_score': 1.198, 'alert': 'False', 'explanation': '3+ !C -- 3+!C; 6-!D; 5-!H; 5-!S; 11-21 HCP'}]
[{'call': '1C', 'insta_score': 1.198, 'alert': 'False', 'explanation': '3+ !C -- 3+!C; 6-!D; 5-!H; 5-!S; 11-21 HCP'}]
[{'call': '1C', 'insta_score': 1.198, 'alert': 'False', 'explanation': '3+ !C -- 3+!C; 6-!D; 5-!H; 5-!S; 11-21 HCP'}]


in the output above:
- `insta_score` reflects what the neural network prefers to bid
- `expected_score` is based on what you expect to score on difference samples (for example, 70% of the time you are making and 30% of the time you are going one off, then the expected score is 0.7 * 620 - 0.3 * 100)

### Samples consistent with the auction so far

Above we said that we are computig expected scores on samples consistent with the auction so far.

How do we get these samples?

We get some samples in the response we get from the bot (i.e `bid.samples`)

In [3]:
bid.samples

[]

each row is one sample. the hands are in the order: N E S W

(West always has the same cards because it's your hand and it's known to you)

In [5]:
# we got some samples above. if we want more, we can always get more

sample_hands, sorted_score, p_hcp, p_shp, good_quality = bot_bid.sample_hands_for_auction(["PAD_START", "PAD_START", "PAD_START", "PASS", "1N", "PASS", "3S","PASS"],0)
print("Good quality:", good_quality)
for i in range(sample_hands.shape[0]):
    print(f'{hand_to_str(sample_hands[i, 0, :], models.n_cards_bidding)} {hand_to_str(sample_hands[i, 1, :], models.n_cards_bidding)} {hand_to_str(sample_hands[i, 2, :], models.n_cards_bidding)} {hand_to_str(sample_hands[i, 3, :], models.n_cards_bidding)}')

Good quality: 0.8637691969672839
xxx.xxxx.Axx.AKx ATxxx.Tx.Txx.Jxx J.KQxx.KQxx.QTxx KQxx.AJx.Jxx.xxx
xxx.xxxx.Axx.AKx QJxxx.Jx.QJx.Jxx A.ATxx.KTxx.Txxx KTxx.KQx.xxx.Qxx
xxx.xxxx.Axx.AKx Axxxxx.Txx.Kx.Tx T.AKQJ.Qxxx.Jxxx KQJ.xx.JTxx.Qxxx
xxx.xxxx.Axx.AKx QJTxx.QJT.KTx.xx x.AKxx.Qxxx.Jxxx AKxx.xx.Jxx.QTxx
xxx.xxxx.Axx.AKx KJxxx.Jx.Kxx.Txx x.AKQx.QTxx.Qxxx AQTx.Txx.Jxx.Jxx
xxx.xxxx.Axx.AKx AQJxxx.xx.Tx.Txx K.KTxx.KQxx.Qxxx Txx.AQJ.Jxxx.Jxx
xxx.xxxx.Axx.AKx Txxxx.AJx.xxx.Qx A.KQxx.KJxx.Jxxx KQJx.Tx.QTx.Txxx
xxx.xxxx.Axx.AKx QJxxxx.xx.QTx.Qx A.AKJT.xxxx.JTxx KTx.Qxx.KJx.xxxx
xxx.xxxx.Axx.AKx QJTxxx.xxx.Jx.Tx x.AKTx.KQxx.QJxx AKx.QJ.Txxx.xxxx
xxx.xxxx.Axx.AKx Axxxx.KJx.xxx.Tx K.QTxx.QJx.QJxxx QJTx.Ax.KTxx.xxx
xxx.xxxx.Axx.AKx Kxxxx.Jxx.Kx.Jxx Q.AKTx.QTxx.Qxxx AJTx.Qx.Jxxx.Txx
xxx.xxxx.Axx.AKx Txxxx.Kxx.xx.Jxx K.AQTx.Kxxx.QTxx AQJx.Jx.QJTx.xxx
xxx.xxxx.Axx.AKx KQxx.KQxx.Txx.Jx A.AJTx.KJxx.xxxx JTxxx.x.Qxx.QTxx
xxx.xxxx.Axx.AKx KJxxx.xx.Qxxx.Jx x.AKQx.JTx.QTxxx AQTx.JTx.Kxx.xxx
xxx.xxxx.Axx.AK